# Setup

**`Runtime` -->  `Change runtime type` --> `GPU`**

In [ ]:
! curl https://getcroc.schollz.com | bash > /dev/null 2>&1
! git clone -q https://github.com/bird-feeder/BirdFSD-YOLOv5.git
%cd BirdFSD-YOLOv5
! pip -q install -r requirements.txt > /dev/null 2>&1
! pip -q install "PyYAML>=5.3.1" wandb > /dev/null 2>&1
! git clone -q https://github.com/ultralytics/yolov5.git

WEIGHTS='' #@param {type: 'string'}

! wget $WEIGHTS -qO best.pt

In [ ]:
#@title Login to W&B
! wandb login

**Run this command on the local server:**


```shell
croc send "BirdFSD-YOLOv5/.env"
```

- Copy the passphrase part and paste it in the cell input below.


In [ ]:
PASSPHRASE = '' #@param {'type': 'string'}
! croc --yes $PASSPHRASE

In [ ]:
#@title Run dataset pre-processing
! python json2yolov5.py -p 1
! mv dataset-YOLO/dataset_config.yml .
! python utils/relative_to_abs.py

# Available Options

```python
arg('--weights', type=str, default=ROOT / 'yolov5s.pt', help='initial weights path')
arg('--cfg', type=str, default='', help='model.yaml path')
arg('--data', type=str, default=ROOT / 'data/coco128.yaml', help='dataset.yaml path')
arg('--hyp', type=str, default=ROOT / 'data/hyps/hyp.scratch-low.yaml', help='hyperparameters path')
arg('--epochs', type=int, default=300)
arg('--batch-size', type=int, default=16, help='total batch size for all GPUs, -1 for autobatch')
arg('--imgsz', '--img', '--img-size', type=int, default=640, help='train, val image size (pixels)')
arg('--rect', action='store_true', help='rectangular training')
arg('--resume', nargs='?', const=True, default=False, help='resume most recent training')
arg('--nosave', action='store_true', help='only save final checkpoint')
arg('--noval', action='store_true', help='only validate final epoch')
arg('--noautoanchor', action='store_true', help='disable AutoAnchor')
arg('--evolve', type=int, nargs='?', const=300, help='evolve hyperparameters for x generations')
arg('--bucket', type=str, default='', help='gsutil bucket')
arg('--cache', type=str, nargs='?', const='ram', help='--cache images in "ram" (default) or "disk"')
arg('--image-weights', action='store_true', help='use weighted image selection for training')
arg('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
arg('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
arg('--single-cls', action='store_true', help='train multi-class data as single-class')
arg('--optimizer', type=str, choices=['SGD', 'Adam', 'AdamW'], default='SGD', help='optimizer')
arg('--sync-bn', action='store_true', help='use SyncBatchNorm, only available in DDP mode')
arg('--workers', type=int, default=8, help='max dataloader workers (per RANK in DDP mode)')
arg('--project', default=ROOT / 'runs/train', help='save to project/name')
arg('--name', default='exp', help='save to project/name')
arg('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
arg('--quad', action='store_true', help='quad dataloader')
arg('--cos-lr', action='store_true', help='cosine LR scheduler')
arg('--label-smoothing', type=float, default=0.0, help='Label smoothing epsilon')
arg('--patience', type=int, default=100, help='EarlyStopping patience (epochs without improvement)')
arg('--freeze', nargs='+', type=int, default=[0], help='Freeze layers: backbone=10, first3=0 1 2')
arg('--save-period', type=int, default=-1, help='Save checkpoint every x epochs (disabled if < 1)')
arg('--local_rank', type=int, default=-1, help='DDP parameter, do not modify')

# Weights & Biases arguments
arg('--entity', default=None, help='W&B: Entity')
arg('--upload_dataset', nargs='?', const=True, default=False, help='W&B: Upload data, "val" option')
arg('--bbox_interval', type=int, default=-1, help='W&B: Set bounding-box image logging interval')
arg('--artifact_alias', type=str, default='latest', help='W&B: Version of dataset artifact to use')
```

# Train

In [ ]:
EPOCHS = 30 #@param {type:"integer"}
BATCH_SIZE = 16 #@param {type:"integer"}
PRETRAINED_WEIGHTS='best.pt'

In [ ]:
# If your training was interrupted for any reason you may continue where you \
# left off using the --resume argument. If your training fully completed, \
# you can start a new training from any model using the --weights argument.

! python yolov5/train.py --img-size 768 --batch $BATCH_SIZE --epochs $EPOCHS \
    --data "dataset_config.yml"  --weights $PRETRAINED_WEIGHTS

# Add metdata

- Click on the run link in the output above.  Paste the run path in the input field, then run the cell.

In [ ]:
import json
import os
import platform

import torch
import torchvision
import wandb

api = wandb.Api()

#@markdown **Run path** `<entity>/<project>/<run_id>`
run_path = '' #@param {type: 'string'}
#@markdown ![run_path](https://d.aibird.me/41a26c3b.png)

! cp "/content/BirdFSD-YOLOv5/dataset-YOLO/classes.txt" .

run = api.run(run_path)
run.upload_file("classes.txt")

torch_version = torch.__version__
torchvision_version = torchvision.__version__
python_version = platform.python_version()
cuda_version = os.popen('nvcc --version | grep release').read().split(', ')[1].split('release ')[1]

version = {'python': python_version, 'cuda': cuda_version,
           'torch': torch_version, 'torchvision': torchvision_version}

run.config.update({'base_ml_framework': version})
run.update()

# Last steps

1. Upload the downloaded weights, then replace the weights link in the first cell with the new link.
2. Clear the `run_path` and `passphrase` fields.
3. Clear the output: `Edit` -> `Clear all output`.